In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from amongus_model import *

# Running the Model + Sensitivity Analysis

In [ ]:
# function to generate the list of variables needed for running the model
def generate_variables_list(x_list, samples, repetitions):

    var_list = []
    
    for x in x_list:        
        var_list.append((samples * repetitions) * [x])
        
    return var_list

In [ ]:
'''
First create the range of the test variables. For instance, we will sample 'sus_kill' for 5 different values 
(steps) and for each value take 3 samples, a sample consists of 3 repetitions the resulting input would be:
'''
repetitions = 1
samples = 2
steps = 5

test_vars = [0, -0.005, -0.01, -0.02, -0.04] 
x_vals = generate_variables_list(test_vars, samples, repetitions)
print(f'This will take {len(x_vals)*len(x_vals[0])} iterations and x_values of {np.unique(x_vals)} as input.')

In [ ]:
'''
=====================
==THIS IS IMPORTANT==
=====================

To select the variable on which the OFAT analysis is to be performed, change the 'selected_variable' to
one of the strings below.

'sigma_3' (sus_kill)
'sigma_4' (sus_group)
'sigma_5' (sus_default)
'gamma_1'
'gamma_2'
'''
selected_variable = 'sigma_3' # <-- Change this!

var_dict = {'sigma_3': 0, 'sigma_4': 1, 'sigma_5': 2, 'gamma_1': 3, 'gamma_2': 4}
ofat_var = var_dict[selected_variable]

# fixed_parameters
starting_positions = [(130, 100), (121, 107), (139, 108), 
(130, 114), (123, 103), (137, 103), (136,112), (124,112)]

# varied parameters
number_of_crewmates = 4
num_tasks_crewmate = 4
injob_time = (70, 100)
impostor_cooldown = 214 # in-game value == 214
impostor_vents = True
just_killed_cooldown = 5
n_iterated_games = 32

# sus matrix parameters
sus_kill = np.inf
sus_vent = np.inf
sus_task = -.05
sus_group = -.01
sus_default = .0005

# trust parameters
gamma1 = 0.04
gamma2 = -0.02

# path to save data to
data_path = f'generated_data/{selected_variable}/{n_iterated_games}'

# Initialize saving lists, MAKE SURE TO BACK UP OLD DATA, WILL BE OVERWRITTEN!!!!
np.save(f'{data_path}/social_matrices/trust_0.npy', np.full((1, number_of_crewmates+1), .5))
np.save(f'{data_path}/misc_data/iteration_data.npy', [])
np.save(f'{data_path}/misc_data/tasks_data.npy', [])
np.save(f'{data_path}/misc_data/crewmates_done_data.npy', [])
np.save(f'{data_path}/misc_data/win_data.npy', [])
np.save(f'{data_path}/misc_data/dead_data.npy', [])

variable_list = [sus_task, sus_group, sus_default, gamma1, gamma2]

for variable in x_vals:
    np.save(f'{data_path}/win_matrices/win_matrix_{variable[0]}_0.npy', np.full((number_of_crewmates+1, 2), 0))
    
    # change only selected variable value
    variable_list[ofat_var] = variable[0]
    
    amongUs = AmongUs('the_skeld', number_of_crewmates, starting_positions, num_tasks_crewmate=num_tasks_crewmate,
    injob_time=injob_time, impostor_cooldown=impostor_cooldown, impostor_vents=impostor_vents,
    just_killed_cooldown=just_killed_cooldown, sus_kill=sus_kill, sus_vent=sus_vent, sus_task=variable_list[0],
    sus_group=variable_list[1], sus_default=variable_list[2], gamma1=variable_list[3], gamma2=variable_list[4],
    n_iterated_games=n_iterated_games, var_name=variable[0], data_path=data_path)
    
    # try except block ensures (rare) errors do not halt data generation
    try:
        amongUs.run(len(variable))
    except:
        continue

# Visualising Results

In this case the results are generated using the 'example_data'. These results can also be generated with the 'generated_data' folder, if the correct configurations are used.

### Winrates for impostor per OFAT variable

In [ ]:
"""
Code which takes as input the win_matrices for the relevant data_sets, and 
plots the resulting winrates for the different social values.
"""

# Set sensitivity values
samples_per_run = 32
amount_of_runs = 5
amount_of_strategies = 4

# Define sigma 3, 4, 5 and gamma 1, 2, and create the correct names for the path
social_list = [[0,-0.025,-0.05,-0.1,-0.2], [0,-0.005,-0.01,-0.02,-0.04], [0,0.00025,0.0005,0.001,0.002], [0,0.02,0.04,0.06,0.08], [0,-0.01,-0.02,-0.03,-0.04]]
social_value = ["sigma_3","sigma_4","sigma_5","gamma_1","gamma_2"]
path = ["32", "1"]

# Run for both the no iteration as the 32 iterations
for iteration in path:
    if iteration == "32":
        title = "no iteration"
    else:
        title = "32 iterations"
        
        
    # Loop through all the values of sigma and gamma
    for l in range(len(social_list)):
        sigma_list = social_list[l]
        variable = social_value[l]
        
        # Loop through all the used strategies
        for k in range(amount_of_strategies):
            strat1_win_list= []
            strat1_error_list = []
            
            # Use the set value for the sigma/gamma variable
            for i in sigma_list:
                prev_value = 0
                win_list = []
                
                # Load the result of the win matrix, and append the relevant wins. ADD THE CORRECT PATH
                for j in range(samples_per_run, amount_of_runs*samples_per_run + samples_per_run ,samples_per_run):
                    tot_wins = np.load(f"example_data/{variable}/{iteration}/win_matrices/win_matrix_{i}_{j}.npy")
                    act_win = tot_wins[k][1] - prev_value
                    prev_value = tot_wins[k][1]
                    win_list.append(act_win)
                
                # Calculate the mean and error of the wins over multiple runs
                avg_wins_strat_1 = (amount_of_strategies/samples_per_run)*np.mean(win_list) 
                error_winst_strat_1 = (amount_of_strategies/samples_per_run)*np.std(win_list)/np.sqrt(amount_of_runs)
                
                strat1_win_list.append(avg_wins_strat_1)
                strat1_error_list.append(error_winst_strat_1)
            
            # Plot the mean with error as shaded area
            plt.plot(sigma_list, strat1_win_list, label = f"strat {k+1}")
            plt.fill_between(sigma_list, np.array(strat1_win_list) + np.array(strat1_error_list), np.array(strat1_win_list) - np.array(strat1_error_list), alpha = 0.2)
        
        # Finish the plot for a set variable with set amount of iterations
        plt.xlabel(f"Value of {variable}")
        plt.ylabel("Average wins")
        plt.title(f"Average winrate for the impostor for {variable}, with {title}")
        plt.ylim(0,0.6)
        plt.xlim(max(sigma_list), min(sigma_list))
        plt.legend()
        plt.savefig(f"plots/Avg_winrate_{variable}", dpi=500)
        plt.show()


### Mean winrate and error bar plot

In [ ]:
"""
Code which takes as input the win_matrices and returns the mean winrate and error
bar plot for the standard settings
"""

# Set the values used in the simulations
samples_per_run = 32
amount_of_runs = 5
amount_of_strategies = 4
strat1_win_list= []
strat1_error_list = []

# Define the standard sigma 3, 4, 5 and gamma 1, 2 and the right path names
social_list = [[-0.1], [-0.01], [0.0005], [0.04], [-0.02]]
social_value = ["sigma_3","sigma_4","sigma_5","gamma_1","gamma_2"]
path = ["32", "1"]
strat_names = ["Aggressive/Active", "Aggressive/Passive", "Careful/Active", "Careful/Passive"]


# Run for both the no iteration as the 32 iterations
for iteration in path:
    if iteration == "32":
        title = "no iteration"
        multiplier = 1
    else:
        title = "32 iterations"
        multiplier = 2
        
    # Loop through all the possible strategies    
    for k in range(amount_of_strategies):
        win_list = []
        label_name = strat_names[k]
        
        # Loop through the values of sigma and gamma
        for l in range(len(social_list)):
            sigma_list = social_list[l]
            variable = social_value[l]
        
            # Loop through the specific values of the social variable and append the wins per strategy
            for i in sigma_list:
                final_results = amount_of_runs*samples_per_run
                tot_wins = np.load(f"example_data/{variable}/{iteration}/win_matrices/win_matrix_{i}_{final_results}.npy")
                act_win = tot_wins[k][1]
                win_list.append(act_win/(8*4))
                 
        # Calculate mean and error
        strat1_win_list.append(np.mean(win_list))
        strat1_error_list.append(np.std(win_list)/(np.sqrt(5)))
     
        
# Transfrom for nice plotting
strats_final = []
error_final = []

for i in range(4):
    strats_final.append([strat1_win_list[i],strat1_win_list[i+4]])
    error_final.append([strat1_error_list[i],strat1_error_list[i+4]])
    
    

        
# Plot the bar
fig, ax = plt.subplots(1,1)

x_ticks_labels = strat_names

ax.grid(axis="y", linestyle = "-", zorder =0)
ax.bar(np.array([0.5,2,3.5,5]), strat1_win_list[0:4], align = "center", width = 0.5, yerr = strat1_error_list[0:4], label = "1 Iteration", ecolor='black', capsize=5, zorder = 3)
ax.bar(np.array([1,2.5,4,5.5]), strat1_win_list[4:8], align = "center", width = 0.5, yerr = strat1_error_list[4:8], label = "32 Iterations", ecolor='black', capsize=5, zorder = 3)

ax.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True)  # labels along the bottom edge are off

plt.ylabel("Mean winrate")
plt.title("Mean winrate for the imposter standard settings")

ax.set_xticks([0.75, 2.25, 3.75, 5.25])

# Set ticks labels for x-axis
ax.set_xticklabels(x_ticks_labels, rotation=0, fontsize=8.5)

plt.ylim(0,0.51)
plt.xlim(0,6)
plt.legend()
plt.savefig("plots/winrates", dpi = 500)
plt.show()

### Change in trust values with different strategies

In [ ]:
"""
Code which calculates the change in trust values for the different strategies,
using the trust matrices as input
"""

# Define sigma 3, 4, 5 and gamma 1, 2
social_list = [[-0.1], [-0.01], [0.0005], [0.04], [-0.02]]
social_value = ["sigma_3","sigma_4","sigma_5","gamma_1","gamma_2"]
strat_names = ["Aggressive/Active", "Aggressive/Passive", "Careful/Active", "Careful/Passive"]
path = ["32", "1"]


# Run for both the no iteration as the 32 iterations
for iteration in path:
    if iteration == "32":
        title = "no iteration"
    else:
        title = "32 iterations"

    trust_list2 = []
    
    
    # Loop through all the social values 
    for l in range(len(social_list)):
        sigma_list = social_list[l]
        variable = social_value[l]
        
        # Loop through the amount of strategies
        for k in range(amount_of_strategies):
            strat1_win_list= []
            strat1_error_list = []
            
            # Pick the exact values of the social variables
            for i in sigma_list:
                prev_value = 0
                
                # For the trust values for each game per sample
                for q in range(0,amount_of_runs):
                    counter = 0
                    trust_list2.append(np.array([0.5,0.5,0.5,0.5,0.5]))
                    
                    # Append the trust value for the relevant strategy
                    for j in range(samples_per_run*q+1, (q+1)*samples_per_run):
                        
                        counter +=1
                        trust = np.load(f"example_data/{variable}/{iteration}/social_matrices/trust_{i}_{j}.npy")
                        act_trust = trust[0]
                        trust_list2.append(act_trust)
 
# Create list with empty lists equal tot he amount of games per run                
trust_AA = [[] for _ in range(samples_per_run)]
trust_AP = [[] for _ in range(samples_per_run)]
trust_CA = [[] for _ in range(samples_per_run)]
trust_CP = [[] for _ in range(samples_per_run)]                
         

# Append the trust values in the correct lists 
values_per_iteration = round(len(trust_list2)/samples_per_run)

for j in range(samples_per_run):
    for i in range(values_per_iteration):   
       index = i*samples_per_run + j
       trust_AA[j].append(trust_list2[index][0])
       trust_AP[j].append(trust_list2[index][1])
       trust_CA[j].append(trust_list2[index][2])
       trust_CP[j].append(trust_list2[index][3])


# Calculate the means and confidence intervals
aa_final = [np.mean(trust_AA[i]) for i in range(samples_per_run)]
aa_error = [1.96*np.std(trust_AA[i])/np.sqrt(values_per_iteration) for i in range(samples_per_run)]

ap_final = [np.mean(trust_AP[i]) for i in range(samples_per_run)]
ap_error = [1.96*np.std(trust_AP[i])/np.sqrt(values_per_iteration) for i in range(samples_per_run)]

ca_final = [np.mean(trust_CA[i]) for i in range(samples_per_run)]
ca_error = [1.96*np.std(trust_CA[i])/np.sqrt(values_per_iteration) for i in range(samples_per_run)]

cp_final = [np.mean(trust_CP[i]) for i in range(samples_per_run)]
cp_error = [1.96*np.std(trust_CP[i])/np.sqrt(values_per_iteration) for i in range(samples_per_run)]
                            

# Plot the figures
plt.figure(1)  
plt.plot(range(samples_per_run), aa_final, label = "Aggressive/Active")
plt.fill_between(range(samples_per_run), np.array(aa_final) + np.array(aa_error), np.array(aa_final) - np.array(aa_error), alpha = 0.2)

plt.plot(range(samples_per_run), ap_final, label = "Aggressive/Passive")
plt.fill_between(range(samples_per_run), np.array(ap_final) + np.array(ap_error), np.array(ap_final) - np.array(ap_error), alpha = 0.2)

plt.plot(range(samples_per_run), ca_final, label = "Careful/Active")
plt.fill_between(range(samples_per_run), np.array(ca_final) + np.array(ca_error), np.array(ca_final) - np.array(ca_error), alpha = 0.2)

plt.plot(range(samples_per_run), cp_final, label = "Careful/Passive")
plt.fill_between(range(samples_per_run), np.array(cp_final) + np.array(cp_error), np.array(cp_final) - np.array(cp_error), alpha = 0.2)


plt.xlabel("Iteration")
plt.ylabel("Average trust score")
plt.title("Mean trust score")
plt.ylim(0.4,0.7)
plt.xlim(0,31)
plt.legend()
plt.savefig("plots/Trust", dpi = 500)
plt.show()      
 


### Violin plot of iterations per strategy

In [ ]:
"""
Code which displays a violin plot of the amount of iterations per strategy,
which uses the iteration_data as input
"""

# Create empty lists for the strategies AA AP CA CP 
game_len_no_iteration = [[],[],[],[]]
game_len_with_iteration = [[],[],[],[]]

# Define the standard sigma 3, 4, 5 and gamma 1, 2
social_list = [[-0.1], [-0.01], [0.0005], [0.04], [-0.02]]
social_value = ["sigma_3","sigma_4","sigma_5","gamma_1","gamma_2"]
path = ["32", "1"]
strat_names = ["Aggressive/Active", "Aggressive/Passive", "Careful/Active", "Careful/Passive"]


# For both itterated and non itterated game, create title and multiplyer for index
for iteration in path:
    if iteration == "32":
        title = "no iteration"
        multiplyer = 1
    else:
        title = "32 iterations"
        multiplyer = 2
        
    # For all the strategies    
    for k in range(amount_of_strategies):
        win_list = []
        label_name = strat_names[k]
        
        # For all the possible social values
        for l in range(len(social_list)):
            sigma_list = social_list[l]
            variable = social_value[l]
            
            # For all the exact values of gamma and sigma, calculate game length and load data
            for i in sigma_list:
                final_results = amount_of_runs*samples_per_run
                tot_len = np.load(f"example_data/{variable}/{iteration}/data_1/iteration_data.npy")
        
        # Create the lists for the strategies for the non-itereated and itereated model
        number_of_points = round(samples_per_run*amount_of_runs*len(social_list)/amount_of_strategies)
                                 
        if title == "no iteration":
            for m in range(number_of_points):
                game_len_no_iteration[k].append(tot_len[4*m + k])
        
        if title == "32 iterations":
            for m in range(number_of_points):
                
                # One run had an error which resulted in 8 runs, another run was done to fill in the value
                # which had a length of 778.
                value = tot_len[4*m + k]
                
                if value == 8:
                    value = 778
                    
                game_len_with_iteration[k].append(value)
            


# Plot the violin plot
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + (q3 - q1) * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value


def set_axis_style(ax, labels):
    ax.get_xaxis().set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Strategy')


fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(9, 4), sharey=True)

ax1.set_title('Non-iterated game')
ax1.set_ylabel("Amount of model steps")
plt.ylim(0,2000)
parts = ax1.violinplot(
        game_len_no_iteration, showmeans=False, showmedians=False,
        showextrema=False)

for pc in parts['bodies']:
    pc.set_facecolor('tab:blue')
    pc.set_edgecolor('None')
    pc.set_alpha(0.5)

quartile1, medians, quartile3 = np.percentile(game_len_no_iteration, [25, 50, 75], axis=1)
whiskers = np.array([
    adjacent_values(sorted_array, q1, q3)
    for sorted_array, q1, q3 in zip(game_len_no_iteration, quartile1, quartile3)])
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]

inds = np.arange(1, len(medians) + 1)
ax1.scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
ax1.vlines(inds, quartile1, quartile3, color='tab:blue', linestyle='-', lw=5)


ax1.grid(axis="y", linestyle = "-", zorder =0)




# Plot the second violin plot
ax2.set_title('32 Iterations')
parts = ax2.violinplot(
        game_len_with_iteration, showmeans=False, showmedians=False,
        showextrema=False)

for pc in parts['bodies']:
    pc.set_facecolor('tab:orange')
    pc.set_edgecolor('None')
    pc.set_alpha(0.5)
    
ax2.grid(axis="y", linestyle = "-", zorder =0)


quartile1, medians, quartile3 = np.percentile(game_len_with_iteration, [25, 50, 75], axis=1)
whiskers = np.array([
    adjacent_values(sorted_array, q1, q3)
    for sorted_array, q1, q3 in zip(game_len_with_iteration, quartile1, quartile3)])
whiskers_min, whiskers_max = whiskers[:, 0], whiskers[:, 1]

inds = np.arange(1, len(medians) + 1)
ax2.scatter(inds, medians, marker='o', color='white', s=30, zorder=3)
ax2.vlines(inds, quartile1, quartile3, color='tab:orange', linestyle='-', lw=5)

# set style for the axes
labels = ["A/A", "A/P", "C/A", "C/P"]
for ax in [ax1, ax2]:
    set_axis_style(ax, labels)

plt.xlabel("Strategy")
plt.ylim(0,2000)
plt.subplots_adjust(bottom=0.15, wspace=0.05)
plt.savefig("plots/Amount of iterations", dpi = 500)
plt.show()